In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
import warnings
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [3]:
from huggingface_hub import notebook_login
notebook_login()

##STT model pipeline

In [4]:
import torch
from transformers import AutomaticSpeechRecognitionPipeline
from transformers import WhisperTokenizer,WhisperForConditionalGeneration,WhisperProcessor
from peft import PeftModel, PeftConfig

stt_model_id = "ShakhzoDavronov/whisper-large-lora-uz"
language = "Uzbek"
task = "transcribe"
stt_config = PeftConfig.from_pretrained(stt_model_id)
stt_model = WhisperForConditionalGeneration.from_pretrained(
    stt_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)

stt_model = PeftModel.from_pretrained(stt_model, stt_model_id)
stt_tokenizer = WhisperTokenizer.from_pretrained(stt_config.base_model_name_or_path, language=language, task=task)
stt_processor = WhisperProcessor.from_pretrained(stt_config.base_model_name_or_path, language=language, task=task)
stt_feature_extractor = stt_processor.feature_extractor
forced_decoder_ids = stt_processor.get_decoder_prompt_ids(language=language, task=task)
stt_pipe = AutomaticSpeechRecognitionPipeline(model=stt_model, tokenizer=stt_tokenizer, feature_extractor=stt_feature_extractor)

def transcribe(audio):
    with torch.cuda.amp.autocast():
        text = stt_pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
    return text

adapter_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
test_audios=['Test_1.wav','Test_2.wav','Test_3.wav','Test_4.wav']
for i,audio in enumerate(test_audios,start=1):
  print(f"Test {i}: {transcribe(audio)}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Test 1: Toshkentda Shavkat Mirziyoyev Rossiya prezidentini kutib oldi
Test 2: Kundan-kunga ahvola yomonlashar edi.
Test 3: Bugun maktabimizda katta tadbir bo‘ldi
Test 4: Hozirgi kunda suniy intellekt bo‘yicha ko‘plab loyihalar qilib kelyapman.


##NER model pipeline

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="peft")

In [4]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig

label_names = ['O', 'B-PERSON', 'B-GPE', 'B-ORG', 'B-LOC', 'B-DATE', 'B-EVENT']
num_labels = len(label_names)
id_to_label = {i: label for i, label in enumerate(label_names)}
label_to_id = {label: i for i, label in enumerate(label_names)}

ner_model_id = "ShakhzoDavronov/xlm-roberta-lora-ner-uz"
ner_config = PeftConfig.from_pretrained(ner_model_id)

ner_model = AutoModelForTokenClassification.from_pretrained(
    ner_config.base_model_name_or_path,
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

ner_model = PeftModel.from_pretrained(ner_model, ner_model_id)
ner_model = ner_model.merge_and_unload()
ner_tokenizer = AutoTokenizer.from_pretrained(ner_config.base_model_name_or_path)

ner_pipe = pipeline('ner', model=ner_model, tokenizer=ner_tokenizer)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [ ]:
text="""Toshkentda Shavkat Mirziyoyev Rossiya prezidentini kutib oldi"""
ner=ner_pipe(text)
for entity in ner:
    print(entity)

{'entity': 'B-LOC', 'score': 0.60946953, 'index': 1, 'word': '▁Toshkent', 'start': 0, 'end': 8}
{'entity': 'B-LOC', 'score': 0.55572474, 'index': 2, 'word': 'da', 'start': 8, 'end': 10}
{'entity': 'B-PERSON', 'score': 0.9300122, 'index': 3, 'word': '▁Shavkat', 'start': 11, 'end': 18}
{'entity': 'B-PERSON', 'score': 0.9212318, 'index': 4, 'word': '▁Mirziyoyev', 'start': 19, 'end': 29}
{'entity': 'B-GPE', 'score': 0.62043226, 'index': 5, 'word': '▁Rossiya', 'start': 30, 'end': 37}


##STT-NER integrated pipeline

In [ ]:
def final_pipeline(audio,ner_pipe,stt_pipe):

  extracted_text=stt_pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
  ner_entities=ner_pipe(extracted_text)
  print(extracted_text)
  for entity in ner_entities:
    print(entity)

final_pipeline('Test_1.wav',ner_pipe,stt_pipe)

Toshkentda Shavkat Mirziyoyev Rossiya prezidentini kutib oldi
{'entity': 'B-LOC', 'score': 0.60946953, 'index': 1, 'word': '▁Toshkent', 'start': 0, 'end': 8}
{'entity': 'B-LOC', 'score': 0.55572474, 'index': 2, 'word': 'da', 'start': 8, 'end': 10}
{'entity': 'B-PERSON', 'score': 0.9300122, 'index': 3, 'word': '▁Shavkat', 'start': 11, 'end': 18}
{'entity': 'B-PERSON', 'score': 0.9212318, 'index': 4, 'word': '▁Mirziyoyev', 'start': 19, 'end': 29}
{'entity': 'B-GPE', 'score': 0.62043226, 'index': 5, 'word': '▁Rossiya', 'start': 30, 'end': 37}
